* Sacred Objectからちゃんとデータを取り出す（普通にDone）
* Sacred Objectを使って，アーチファクトを取り出す（Artifactは，正直形式がわからないので使わない．_run['info']['log_dir']を記録しておいて，普通にファイルパスで読み取る
* 既にやってた実験をSacredの方に入れ込む方法を考える
  (これはSacredのUI使う方が多分やりやすい）

In [23]:
from sacred_wrap import SacredRecords
from sacred_wrap import MongoExtractor

Signature:
SacredRecords.from_mongo(
    query_dict,
    url=None,
    db_name=None,
    fields=None,
    exact_search=False,
    status='COMPLETED',
)
Docstring: <no docstring>
File:      /share/iwasawa/constrastive_predictive_coding/sacred_wrap.py
Type:      method


In [26]:
df = SacredRecords.from_mongo({'method.name': 'CPC', 'seed': 123456}, db_name='CPC_test')
df

,dataset.K,dataset.L,dataset.name,dataset.test_domain,dataset.validation,db_name,gpu,method.context,method.hidden,method.name,...,valid-loss-3,valid-loss-4,train-accuracy-5,train-accuracy-6,train-loss-5,train-loss-6,valid-accuracy-5,valid-accuracy-6,valid-loss-5,valid-loss-6
0,5,12,oppG,S1,ADL4-ADL5,CPC_test,3,200,1600,CPC,...,"[0.45371005819602445, inf, 0.302120764316483, ...","[0.4801474826579744, inf, 0.3308345401151614, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,12,oppG,S1,ADL4-ADL5,CPC_test,2,400,1600,CPC,...,"[0.45319285921075125, 0.3394818873229352, 0.29...","[0.4713144922121005, inf, 0.33573758686808025,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,12,oppG,S1,ADL4-ADL5,CPC_test,1,800,1600,CPC,...,"[0.44412956860932434, inf, inf, 0.265691587532...","[0.472256811843677, inf, inf, 0.29690771668472...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,12,oppG,S1,ADL4-ADL5,CPC_test,0,1600,1600,CPC,...,"[0.4381232976236127, inf, 0.3113577083091844, ...","[0.4644385041161017, 0.35976594212380325, 0.34...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,12,oppG,S1,ADL4-ADL5,CPC_test,2,200,400,CPC,...,"[0.5482556579465215, 0.40724434940652415, 0.34...","[0.5603910054672848, 0.43406626683744515, 0.36...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,12,oppG,S1,ADL4-ADL5,CPC_test,3,200,800,CPC,...,"[0.49599879810755904, 0.3647491203790361, 0.31...","[0.5111820735037327, 0.3894911909645254, 0.341...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,12,oppG,S1,ADL4-ADL5,CPC_test,1,200,3200,CPC,...,"[0.4101121229204265, 0.32619875771078194, 0.29...","[0.43427001617171546, 0.35239154371348297, 0.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,12,oppG,S1,ADL4-ADL5,CPC_test,0,800,1600,CPC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,12,oppG,S1,ADL4-ADL5,CPC_test,0,800,1600,CPC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,7,12,oppG,S1,ADL4-ADL5,CPC_test,2,800,1600,CPC,...,"[0.46055375615304167, 0.3349308127706701, 0.27...","[0.4711907143619927, 0.3634863597425548, 0.299...","[0.7639180672268907, 0.8484933035714286, 0.881...","[0.7540211397058824, 0.8396303834033614, 0.875...","[0.4873472778987484, 0.35292161788259235, 0.28...","[0.5016939223063093, 0.3733990410295855, 0.297...","[0.7632723721590909, 0.8330522017045454, 0.867...","[0.7551935369318182, 0.8193359375, 0.856844815...","[0.4856484572995793, 0.3809159377759153, 0.322...","[0.5007589523765174, 0.4054652526974678, 0.338..."


### 学習用のスクリプト

In [31]:
from datasets import OppG
from utils import split_dataset


def get_dataset(name, validation, test_domain, L, K):
    """Prepare datasets for train, valid and test with configurations.

    Parameter
    ---------
    name : str
    validation : str or list
    test_domain : str or list
    L : int
    K : int

    Return
    ------
    {train/valid}_dataset_{joint/marginal} : torch.Dataset
        {train/valid} datasets from {joint/marginal} distributions
    """
    if isinstance(validation, str):
        validation = validation.split('-')
    all_adls = OppG.get('all_adls')
    all_domain = OppG.get('all_domain_key')
    train_adls = sorted(list(set(all_adls) - set(validation)))
    train_domain = sorted(list(set(all_domain) - set([test_domain])))
    train_dataset_joint = OppG(
        train_domain, l_sample=30, interval=15, T=K+L, adl_ids=train_adls)
    valid_dataset_joint = OppG(
        train_domain, l_sample=30, interval=15, T=K+L, adl_ids=validation)

    # marginal sample come from same datasets for simplicity
    # Same train-valid split with joint dataset
    train_dataset_marginal = OppG(
        train_domain, l_sample=30, interval=15, T=K, adl_ids=train_adls)
    valid_dataset_marginal = OppG(
        train_domain, l_sample=30, interval=15, T=K, adl_ids=validation)
    test_dataset = OppG(test_domain, l_sample=30, interval=15, T=K+L)
    
    return train_dataset_joint, valid_dataset_joint, train_dataset_marginal, valid_dataset_marginal, test_dataset


In [32]:
from utils import flatten_dict
extractor = MongoExtractor(None, 'CPC_test')
query = {'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 200, 'num_gru': 1}, 'optim': {'lr': 0.0001, 'num_batch': 100, 'batch_size': 128}}
query = flatten_dict(query)
for result in extractor.find(query, ['config', 'info'], False, 'COMPLETED'):
    print("ok")

ok


In [33]:
from datasets import OppG
from opportunity import Encoder, ContextEncoder, Predictor

In [34]:
_config = result['config']
classifier = {
    'pretrain': True, 
    'finetune_g': False, 
    'use_c_enc': False, 
    'finetune_c': False, 
}

classifier_optim = {
    'lr': 0.001, 
    'num_batch': 10000, 
    'batch_size': 128,
    'monitor_per': 100, 
}
_config['classifier'] = classifier
_config['classifier_optim'] = classifier_optim

In [35]:
import os
import datetime
from copy import deepcopy

import torch
from torch import nn
from torch import optim
from torch.utils import data
from tensorboardX import SummaryWriter

from run_sacred import get_model, get_dataset

In [36]:
from label_predict import Classifier

def get_classifier(model, finetune_g, use_c_enc, finetune_c, **kwargs):
    if use_c_enc:
        classifier = Classifier(
            num_classes=train_dataset.get('num_classes'),
            g_enc=model.g_enc, c_enc=model.c_enc, finetune_g=finetune_g, finetune_c=finetune_c).cuda()
    else:
        classifier = Classifier(
            num_classes=train_dataset.get('num_classes'),
            g_enc=model.g_enc, finetune_g=finetune_g).cuda()
    return classifier

In [37]:
train_dataset, valid_dataset, _, _, test_dataset = get_dataset(**_config['dataset'])
train_loader = data.DataLoader(train_dataset, batch_size=_config['classifier_optim']['num_batch'], shuffle=True)


In [38]:
model = get_model(input_shape=train_dataset.get('input_shape'), K=_config['dataset']['K'], **_config['method'])
if _config['classifier']['pretrain']:
    search_config = deepcopy(_config)
    del search_config['classifier']
    del search_config['classifier_optim']
    extractor = MongoExtractor(None, _config['db_name'])
    # TODO: Should check whether the len(list) is one or not
    result = list(extractor.find(search_config, ['config', 'info'], False, 'COMPLETED'))[0]
    path = os.path.join(result['info']['log_dir'], 'model_{}.pth'.format(_config['optim']['num_batch']))
    model.load_state_dict(torch.load(path))

1600 200


In [39]:
model

CPCModel(
  (g_enc): Encoder(
    (feature): Sequential(
      (0): Conv2d(1, 50, kernel_size=(1, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(50, 40, kernel_size=(1, 5), stride=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(40, 20, kernel_size=(1, 3), stride=(1, 1))
      (7): ReLU()
      (8): Dropout(p=0.5)
      (9): Flatten()
      (10): Linear(in_features=4520, out_features=1600, bias=True)
      (11): ReLU()
      (12): Dropout(p=0.5)
    )
  )
  (c_enc): ContextEncoder(
    (gru): GRU(1600, 200)
  )
  (predictor): Predictor(
    (linears): ModuleList(
      (0): Linear(in_features=200, out_features=1600, bias=True)
      (1): Linear(in_features=200, out_features=1600, bias=True)
      (2): Linear(in_features=200, out_features=1600, bias=True)
      (3): Linear(in_features=200, out_f

### calculate domain invariance

In [96]:
import itertools
from torch.utils import data


def l2diff(x1, x2):
    """
    standard euclidean norm
    """
    return ((x1-x2)**2).sum().sqrt()


def moment_diff(sx1, sx2, k):
    """
    difference between moments
    """
    ss1 = (sx1**k).mean(0)
    ss2 = (sx2**k).mean(0)
    return l2diff(ss1, ss2)


class CMD(object):
    def __init__(self, n_moments=5):
        self.n_moments = n_moments

    def __call__(self, x1, x2):
        mx1 = x1.mean(dim=0)
        mx2 = x2.mean(dim=0)
        sx1 = x1 - mx1
        sx2 = x2 - mx2
        dm = l2diff(mx1, mx2)
        scms = dm

        for i in range(self.n_moments-1):
            # moment diff of centralized samples
            scms += moment_diff(sx1, sx2, i+2)
        return scms


def pairwise_divergence(datasets, func, criterion, batch_size=128, num_batch=None):
    divergence = 0
    num_total_batch = 0
    if num_batch is None:
        num_batch = max([len(dataset) for dataset in datasets])

    for dataset1, dataset2 in itertools.combinations(datasets, 2):
        loader1 = data.DataLoader(dataset1, shuffle=False, batch_size=batch_size)
        loader2 = data.DataLoader(dataset2, shuffle=False, batch_size=batch_size)

        for num_iter, ((X1, Y1), (X2, Y2)) in enumerate(zip(loader1, loader2)):
            divergence += criterion(func(X1.float().cuda()), func(X2.float().cuda())).item()
            if ((num_iter+1) % num_batch) == 0:
                break
        num_total_batch += (num_iter+1)
    return divergence/num_total_batch


def get_feature_of(g_enc, c_enc, L):
    def _func(X):
        if c_enc is None:
            return g_enc(X[..., L-1])
        else:
            return get_context(X[..., :(L-1)], g_enc, c_enc)
    return _func

In [97]:
divergence_criterion = CMD(n_moments=5)
get_g_of = get_feature_of(model.g_enc, None, _config['dataset']['L'])

In [101]:
%time pairwise_divergence(train_dataset.datasets[:2], get_g_of, divergence_criterion, num_batch=10)

CPU times: user 35.5 s, sys: 676 ms, total: 36.2 s
Wall time: 4.47 s


4.290335929393768

In [43]:
def label_predict(
        L, K, g_enc_size, context_size, num_gru, pretrain,  # parameters for model
        finetune_g, use_c_enc=False, finetune_c=False,
        num_batch=10000, iteration_at=10000):
    """Train label classifier.

    TODO: should be separate out the data loading function and classifier building function.
    This function should be focus on the training given classifier with given configurations
    (like datasets or hyperparameters). The name might be just a train() then.

    The ideal argments migth be similar to validate_label_prediction function, with additional
    arguments for hyperparameters.

    Parameter
    ---------
    pretrain : bool
        wheter use CPC pretrained model.
    finetune_g : bool
        wheter train g_encoder.
    use_c_func : bool (default False)
        wheter use context encoder (autoregressive encoder summarize several inputs).
    finetune_c : bool (default False)
        wheter train context encoder. It should be false if use_c_func is False, and True
        if use_c_func and finetune_g is both True.


    Example
    -------
    Case1: pretrain=True and finetune_g=False
    => Shallow classifier over unsupervisly learned representations
    Case2: pretrain=False and finetune_g=True
    => Fully supervised learning
    Case3: pretrain=True and finetune_g=True
    => Unsup+Sup learning
    Case4: pretrain=False and finetune_g=False
    => Baseline with random representations (to clarify the effect of CPC, not an architecture)
    """
    if not use_c_enc and finetune_c:
        print("Invalid combination of parameters")
        return None
    if use_c_enc and finetune_g and not finetune_c:
        print("Invalid combination of parameters", use_c_enc, finetune_g, finetune_c)
        return None

    # Load dataset
    # TODO: it should be easy to change the dataset
    all_adls = ['Drill', 'ADL1', 'ADL2', 'ADL3', 'ADL4', 'ADL5']
    valid_adls = ['ADL1', 'ADL2', 'ADL3', 'ADL4', 'ADL5']
    train_adls = list(set(all_adls) - set(valid_adls))
    dataset_name = '-'.join(valid_adls)
    print("Load datasets ...")
    train_dataset = OppG(
        'S2,S3,S4', 'Gestures', l_sample=30, interval=15, T=K+L, adl_ids=train_adls)
    valid_dataset = OppG(
        'S2,S3,S4', 'Gestures', l_sample=30, interval=15, T=K+L, adl_ids=valid_adls)
    train_loader_joint = data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_dataset = OppG('S1', 'Gestures', l_sample=30, interval=15, T=K+L)

    print("Train: {}, Valid: {}, Test: {}".format(
        len(train_dataset), len(valid_dataset), len(test_dataset))
    )

    folder_name = BASE_PATH.format(g_enc_size, context_size, num_gru)
    folder_name = '{}/{}-{}'.format(folder_name, L, K)
    # save_folder_name = '{}/{}-{}/{}'.format(folder_name, L, K, dataset_name)
    # print(save_folder_name)
    os.makedirs(folder_name, exist_ok=True)
    monitor_per = 100  # output the result per monitor_each iterations

    # parameter of label train
    g_enc = Encoder(input_shape=train_dataset.get('input_shape'), hidden_size=g_enc_size).cuda()
    c_enc = ContextEncoder(input_shape=g_enc.output_shape(), num_layers=num_gru, hidden_size=context_size).cuda()
    predictor = Predictor((None, c_enc.hidden_size), g_enc.output_shape()[1], max_steps=K).cuda()
    model = CPCModel(g_enc, c_enc, predictor).cuda()
    if pretrain:
        model.load_state_dict(torch.load('{}-{}.pth'.format(folder_name, iteration_at)))

    if use_c_enc:
        classifier = Classifier(
            num_classes=train_dataset.get('num_classes'),
            g_enc=g_enc, c_enc=c_enc, finetune_g=finetune_g, finetune_c=finetune_c).cuda()
    else:
        classifier = Classifier(
            num_classes=train_dataset.get('num_classes'),
            g_enc=g_enc, finetune_g=finetune_g).cuda()
    print(classifier)
    # optimizer = optim.Adam(classifier.parameters(), lr=0.001)
    optimizer = optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.NLLLoss()

    train_results = []
    valid_results = []
    test_results = []
    folder_name = '{}/{}/label_predict'.format(dataset_name, folder_name)
    os.makedirs(folder_name, exist_ok=True)
    for num_iter in range(num_batch):
        optimizer.zero_grad()
        X, Y = train_loader_joint.__iter__().__next__()
        y = Y[:, 0, L-1].long().cuda()
        pred_y = classifier(X[..., :L].float().cuda())
        loss = criterion(pred_y, y)
        loss.backward()
        optimizer.step()

        if ((num_iter + 1) % monitor_per) != 0:
            continue
        print(num_iter+1)
        train_results.append(validate_label_prediction(classifier, train_dataset, L=L, nb_batch=None))
        valid_results.append(validate_label_prediction(classifier, valid_dataset, L=L, nb_batch=None))
        print('train', train_results[-1]['accuracy'], train_results[-1]['f1macro'])
        print('valid', valid_results[-1]['accuracy'], valid_results[-1]['f1macro'])
        print('test', test_results[-1]['accuracy'], test_results[-1]['f1macro'])
        test_results.append(validate_label_prediction(classifier, test_dataset, L=L, nb_batch=None))
        pd.DataFrame(train_results).to_csv(
            os.path.join(folder_name, '{}-{}-{}-{}-train.csv'.format(pretrain, finetune_g, use_c_enc, finetune_c)))
        pd.DataFrame(valid_results).to_csv(
            os.path.join(folder_name, '{}-{}-{}-{}-valid.csv'.format(pretrain, finetune_g, use_c_enc, finetune_c)))
        pd.DataFrame(test_results).to_csv(
            os.path.join(folder_name, '{}-{}-{}-{}-test.csv'.format(pretrain, finetune_g, use_c_enc, finetune_c)))


In [53]:
from fastprogress import master_bar, progress_bar
from time import sleep
mb = master_bar(range(10))
for i in mb:
    for j in progress_bar(range(100), parent=mb):
        sleep(0.1)
        mb.child.comment = f'second bar stat'
    mb.first_bar.comment = f'first bar stat'
    mb.write(f'Finished loop {i}.')
    #mb.update_graph(graphs, x_bounds, y_bounds)

Finished loop 0.
Finished loop 1.
Finished loop 2.
Finished loop 3.
Finished loop 4.
Finished loop 5.
Finished loop 6.
Finished loop 7.
Finished loop 8.
Finished loop 9.


In [45]:
import numpy as np
mb = master_bar(range(10))
mb.names = ['cos', 'sin']
for i in mb:
    for j in progress_bar(range(100), parent=mb):
        if j%10 == 0:
            k = 100 * i + j
            x = np.arange(0, 2*k*np.pi/1000, 0.01)
            y1, y2 = np.cos(x), np.sin(x)
            graphs = [[x,y1], [x,y2]]
            x_bounds = [0, 2*np.pi]
            y_bounds = [-1,1]
            mb.update_graph(graphs, x_bounds, y_bounds)
            mb.child.comment = f'second bar stat'
    mb.first_bar.comment = f'first bar stat'
    mb.write(f'Finished loop {i}.')

Finished loop 0.
Finished loop 1.
Finished loop 2.
Finished loop 3.
Finished loop 4.
Finished loop 5.
Finished loop 6.
Finished loop 7.
Finished loop 8.
Finished loop 9.


In [54]:
from tqdm import tqdm
import time
from collections import OrderedDict
from random import random

def func(c):
    # とりあえず入力文字をそのまま返してみようと思った。
    return c

with tqdm(list("abcdefg")) as pbar:
    for i, char in enumerate(pbar):
        sleep(0.1)
        pbar.set_postfix(OrderedDict(
            output=func(char), loss=random(), acc=random(), message='okkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk'))
        time.sleep(0.01)

100%|██████████| 7/7 [00:00<00:00,  8.95it/s, output=g, loss=0.563, acc=0.858, message=okkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk]


In [55]:
from collections import OrderedDict


In [56]:
dic = OrderedDict()
dic['acc'] = 0.8
dic['loss'] = 0.23

SyntaxError: unexpected EOF while parsing (<ipython-input-58-91c4ef0e72fc>, line 1)

In [63]:
text = ''
text = ', '.join(['{}:{:.3f}'.format(k, v) for k, v in dic.items()])


In [64]:
print(text)

acc:0.800, loss:0.230
